<a href="https://colab.research.google.com/github/simmighosh01/OHSL_SearchEngine_ChatBot/blob/main/SearchEngineChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

download_folder = 'downloaded_images/'
visited_urls = set()
output_file = 'scraped_content.txt'

def save_image(image_url, folder):
    try:
        img_response = requests.get(image_url)
        if img_response.status_code == 200:
            img_name = os.path.basename(urlparse(image_url).path)
            img_path = os.path.join(folder, img_name)
            with open(img_path, 'wb') as img_file:
                img_file.write(img_response.content)
            return img_name
        else:
            print(f"Failed to retrieve image. Status code: {img_response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while saving image: {e}")
        return None

def print_element(element, url, level=0):
    indent = '    ' * level
    with open(output_file, 'a', encoding='utf-8') as f:
        if element.name == 'h1':
            f.write(f"\n# {element.get_text()}\n")
        elif element.name == 'h2':
            f.write(f"\n## {element.get_text()}\n")
        elif element.name == 'h3':
            f.write(f"\n### {element.get_text()}\n")
        elif element.name == 'p':
            f.write(f"{indent}{element.get_text()}\n")
        elif element.name == 'ul':
            for li in element.find_all('li'):
                f.write(f"{indent} - {li.get_text()}\n")
        elif element.name == 'img':
            img_url = urljoin(url, element.get('src'))
            img_name = save_image(img_url, download_folder)
            if img_name:
                img_path = os.path.join(download_folder, img_name)
                f.write(f"{indent}![{element.get('alt', 'Image')}]({img_path})\n")
        elif element.name == 'table':
            for row in element.find_all('tr'):
                cols = row.find_all(['th', 'td'])
                col_texts = [col.get_text(strip=True) for col in cols]
                f.write(indent + ' | '.join(col_texts) + '\n')

def scrape_page(url, base_url, download_folder='downloaded_images', level=0):
    if url in visited_urls:
        return
    visited_urls.add(url)

    try:
        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            main_content = soup.find('div', {'class': 'main-content'})
            if not main_content:
                main_content = soup.find('div', {'class': 'content'})

            if main_content:
                for element in main_content.find_all(['h1', 'h2', 'h3', 'p', 'ul', 'img', 'table', 'a']):
                    print_element(element, url, level)
            else:
                print(f"Main content not found on {url}. Please check the class name used to identify the main content.")

            for link in soup.find_all('a', href=True):
                link_url = urljoin(base_url, link['href'])
                if base_url in link_url and link_url not in visited_urls:
                    scrape_page(link_url, base_url, download_folder, level + 1)
        else:
            print(f"Failed to retrieve the website. Status code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while retrieving {url}: {e}")

project_url = input("Enter the URL of the project page: ").strip()

if project_url.startswith('http://') or project_url.startswith('https://'):
    base_url = "{0.scheme}://{0.netloc}".format(urlparse(project_url))

    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(f"Scraping website: {project_url}\n\n")

    scrape_page(project_url, base_url)
else:
    print("Invalid URL. Please ensure the URL starts with http:// or https://")

Enter the URL of the project page: https://ohsl.us
Failed to retrieve the website. Status code: 404


/usr/lib/python3.10/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


Main content not found on https://ohsl.us/rss.xml. Please check the class name used to identify the main content.
Main content not found on https://ohsl.us/sites/default/files/2020-08/Poster%20November%2019%20Background.pdf. Please check the class name used to identify the main content.
Main content not found on https://ohsl.us/sites/default/files/2020-08/Tony%20Joseph%20Poster.pdf. Please check the class name used to identify the main content.
Main content not found on https://ohsl.us/sites/default/files/2020-08/Philip%20Bourne%20Announcement.pdf. Please check the class name used to identify the main content.
Main content not found on https://ohsl.us/sites/default/files/2020-08/Buetow_Announcement.pdf. Please check the class name used to identify the main content.
Failed to retrieve the website. Status code: 404
Failed to retrieve the website. Status code: 404
Failed to retrieve the website. Status code: 404
Main content not found on https://ohsl.us/sites/default/files/2022-03/Explain

In [2]:
!pip install gradio
!pip install pypdf
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install sentence_transformers
!pip install llama_index
%pip install llama-index-llms-huggingface
%pip install llama-index-embeddings-langchain
!pip install -U langchain-community

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 116.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━

In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Document
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import PromptTemplate
import torch
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.embeddings.langchain import LangchainEmbedding
import gradio as gr

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [4]:
# Read the scraped content from the .txt file
def load_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Load the scraped content into a Document
text_content = load_text_file('scraped_content.txt')
documents = [Document(text=text_content)]

system_prompt = """
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""

# Default format supportable by LLama2
query_wrapper_prompt = PromptTemplate("{query_str}")

# Load the model with optimized settings
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="daryl149/llama-2-7b-chat-hf",
    model_name="daryl149/llama-2-7b-chat-hf",
    device_map="auto",  # Use device_map for automatic device placement
    model_kwargs={"torch_dtype": torch.float16, "load_in_8bit": True}
)

# Use an efficient embedding model
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))

service_context = ServiceContext.from_defaults(
    chunk_size=512,  # Reduced chunk size
    llm=llm,
    embed_model=embed_model
)

index = VectorStoreIndex.from_documents(documents, service_context=service_context)

query_engine = index.as_query_engine()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-4-777c8601b592>:34: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [5]:
# Function to handle chat queries and return responses
def chat(query, history=[]):
    response = query_engine.query(query)
    history.append((query, str(response)))
    return history, history

# Gradio interface
with gr.Blocks(css=".gradio-container {height: 100vh !important}") as demo:
    gr.Markdown("# Simple Chatbot", elem_id="title")
    chatbot = gr.Chatbot()
    query = gr.Textbox(placeholder="Enter your query here...", show_label=False)
    submit = gr.Button("Submit")

    def submit_query(query, history):
        return chat(query, history)

    submit.click(submit_query, [query, chatbot], [chatbot, chatbot])

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f20912cdca55c80dd6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [6]:
# Query the index
response = query_engine.query("what is project goals")
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



The project aims to create an AI-Powered Sign Language Virtual Assistant to address the challenges faced by deaf or mute individuals in utilizing voice-based virtual assistants. The goal of the project is to develop an interface that will help the Deaf-mutes to use these Virtual Assistants easily. Designing such an interface will make them find their freedom while using such technologies and might boost their confidence in this Digital Age.
